<a href="https://colab.research.google.com/github/harnalashok/classification/blob/main/7_xgboost_housingprices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 30th May, 2023
# Ref: https://www.kaggle.com/code/alexisbcook/xgboost/tutorial
# Dataset: melb_data.csv

In this tutorial, you will learn how to build and optimize models with **gradient boosting**.  This method dominates many Kaggle competitions and achieves state-of-the-art results on a variety of datasets. 

# Introduction

For much of this course, you have made predictions with the random forest method, which achieves better performance than a single decision tree simply by averaging the predictions of many decision trees.

We refer to the random forest method as an "ensemble method".  By definition, **ensemble methods** combine the predictions of several models (e.g., several trees, in the case of random forests).  

Next, we'll learn about another ensemble method called gradient boosting.  

# Gradient Boosting

**Gradient boosting** is a method that goes through cycles to iteratively add models into an ensemble.  

It begins by initializing the ensemble with a single model, whose predictions can be pretty naive.  (Even if its predictions are wildly inaccurate, subsequent additions to the ensemble will address those errors.)

Then, we start the cycle:
- First, we use the current ensemble to generate predictions for each observation in the dataset.  To make a prediction, we add the predictions from all models in the ensemble.  
- These predictions are used to calculate a loss function (like [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error), for instance).
- Then, we use the loss function to fit a new model that will be added to the ensemble.  Specifically, we determine model parameters so that adding this new model to the ensemble will reduce the loss.  (*Side note: The "gradient" in "gradient boosting" refers to the fact that we'll use [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) on the loss function to determine the parameters in this new model.*)
- Finally, we add the new model to ensemble, and ...
- ... repeat!

![tut6_boosting](https://storage.googleapis.com/kaggle-media/learn/images/MvCGENh.png)


# Example

We begin by loading the training and validation data in `X_train`, `X_valid`, `y_train`, and `y_valid`. 

In [18]:
# 1.0 Call libraries:

import pandas as pd
from  pathlib import Path

# 1.0.1
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 1.0.2
from xgboost import XGBRegressor

In [2]:
# 1.1 Gdrive:

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [23]:
# 1.2 Display from a cell outputs of multiple commands:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
# 1.3 Path:

pathToFolder = "/gdrive/MyDrive/Colab_data_files/melb_data" 
path = Path(pathToFolder) / "melb_data.csv"

In [21]:
# 2.0 Read the data
data = pd.read_csv(path)


In [24]:
# 2.1 Show data:

data.shape   # (13580, 21)
data.head()

(13580, 21)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [25]:
# 2.2 Select subset of predictors:

cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# 2.3 Select target
y = data.Price

# 2.4 Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In this example, you'll work with the XGBoost library.  **XGBoost** stands for **extreme gradient boosting**, which is an implementation of gradient boosting with several additional features focused on performance and speed.  (_Scikit-learn has another version of gradient boosting, but XGBoost has some technical advantages._) 

In the next code cell, we import the scikit-learn API for XGBoost ([`xgboost.XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)).  This allows us to build and fit a model just as we would in scikit-learn.  As you'll see in the output, the `XGBRegressor` class has many tunable parameters -- you'll learn about those soon!

In [26]:
# 3.0
my_model = XGBRegressor()
my_model.fit(X_train, y_train, verbose = True)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

We also make predictions and evaluate the model.

In [8]:
# 3.1 Predictions:

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 236339.49751242637


# Parameter Tuning

XGBoost has a few parameters that can dramatically affect accuracy and training speed.  The first parameters you should understand are:

### `n_estimators`
`n_estimators` specifies how many times to go through the modeling cycle described above.  It is equal to the number of models that we include in the ensemble. 

- Too _low_ a value causes _underfitting_, which leads to inaccurate predictions on both training data and test data. 
- Too _high_ a value causes _overfitting_, which causes accurate predictions on training data, but inaccurate predictions on test data (_which is what we care about_). 

Typical values range from 100-1000, though this depends a lot on the `learning_rate` parameter discussed below.

Here is the code to set the number of models in the ensemble:

In [19]:
# 3.2 Set n_estimators:

my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, verbose = True)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

### `early_stopping_rounds`

`early_stopping_rounds` offers a way to automatically find the ideal value for `n_estimators`. Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for `n_estimators`.  It's smart to set a high value for `n_estimators` and then use `early_stopping_rounds` to find the optimal time to stop iterating.

Since random chance sometimes causes a single round where validation scores don't improve, you need to specify a number for how many rounds of straight deterioration to allow before stopping.  Setting `early_stopping_rounds=5` is a reasonable choice.  In this case, we stop after 5 straight rounds of deteriorating validation scores.

When using `early_stopping_rounds`, you also need to set aside some data for calculating the validation scores - this is done by setting the `eval_set` parameter.  

We can modify the example above to include early stopping:

In [13]:
# 3.3 Set early stop rounds:

my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose= True)

[0]	validation_0-rmse:932999.23746
[1]	validation_0-rmse:722524.48524
[2]	validation_0-rmse:588820.33413
[3]	validation_0-rmse:506012.90428
[4]	validation_0-rmse:459015.64219
[5]	validation_0-rmse:429630.04522
[6]	validation_0-rmse:413554.71344
[7]	validation_0-rmse:405083.67137


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[8]	validation_0-rmse:400551.46223
[9]	validation_0-rmse:396260.63952
[10]	validation_0-rmse:390998.82190
[11]	validation_0-rmse:390045.48076
[12]	validation_0-rmse:387673.09606
[13]	validation_0-rmse:387220.55035
[14]	validation_0-rmse:385375.71052
[15]	validation_0-rmse:384296.45124
[16]	validation_0-rmse:382875.02127
[17]	validation_0-rmse:382492.04396
[18]	validation_0-rmse:382459.77773
[19]	validation_0-rmse:380374.94809
[20]	validation_0-rmse:379751.21488
[21]	validation_0-rmse:377638.64248
[22]	validation_0-rmse:376534.68056
[23]	validation_0-rmse:376689.02626
[24]	validation_0-rmse:375978.35272
[25]	validation_0-rmse:375972.86418
[26]	validation_0-rmse:375748.74039
[27]	validation_0-rmse:375256.63792
[28]	validation_0-rmse:374313.78411
[29]	validation_0-rmse:374068.40582
[30]	validation_0-rmse:371772.23791
[31]	validation_0-rmse:371674.05786
[32]	validation_0-rmse:371571.00679
[33]	validation_0-rmse:371272.31612
[34]	validation_0-rmse:371157.31686
[35]	validation_0-rmse:370738.

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

If you later want to fit a model with all of your data, set `n_estimators` to whatever value you found to be optimal when run with early stopping.

### `learning_rate`

Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the **learning rate**) before adding them in.  

This means each tree we add to the ensemble helps us less.  So, we can set a higher value for `n_estimators` without overfitting.  If we use early stopping, the appropriate number of trees will be determined automatically.

In general, a small learning rate and large number of estimators will yield more accurate XGBoost models, though it will also take the model longer to train since it does more iterations through the cycle.  As default, XGBoost sets `learning_rate=0.1`.

Modifying the example above to change the learning rate yields the following code:

In [12]:
# 3.4 learning_rate:

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose= True)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:1195490.17662
[1]	validation_0-rmse:1144269.13722
[2]	validation_0-rmse:1096065.83039
[3]	validation_0-rmse:1050779.90953
[4]	validation_0-rmse:1008011.72305
[5]	validation_0-rmse:967450.65815
[6]	validation_0-rmse:929524.80330
[7]	validation_0-rmse:893629.17829
[8]	validation_0-rmse:860039.20964
[9]	validation_0-rmse:828368.90103
[10]	validation_0-rmse:798476.08492
[11]	validation_0-rmse:770552.77563
[12]	validation_0-rmse:743942.04177
[13]	validation_0-rmse:719332.84291
[14]	validation_0-rmse:696050.29498
[15]	validation_0-rmse:674334.22360
[16]	validation_0-rmse:653999.12453
[17]	validation_0-rmse:634713.55912
[18]	validation_0-rmse:617067.55474
[19]	validation_0-rmse:600456.65184
[20]	validation_0-rmse:585154.95674
[21]	validation_0-rmse:570511.08907
[22]	validation_0-rmse:557257.13236
[23]	validation_0-rmse:544530.72213
[24]	validation_0-rmse:532991.79233
[25]	validation_0-rmse:522224.56455
[26]	validation_0-rmse:512465.95072
[27]	validation_0-rmse:502651.608

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

### `n_jobs`
On larger datasets where runtime is a consideration, you can use parallelism to build your models faster.  It's common to set the parameter `n_jobs` equal to the number of cores on your machine.  On smaller datasets, this won't help. 

The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the `fit` command.

Here's the modified example:

In [14]:
# n_jobs:

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose= True)

[0]	validation_0-rmse:1195490.17662
[1]	validation_0-rmse:1144269.13722
[2]	validation_0-rmse:1096065.83039
[3]	validation_0-rmse:1050779.90953
[4]	validation_0-rmse:1008011.72305
[5]	validation_0-rmse:967450.65815


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-rmse:929524.80330
[7]	validation_0-rmse:893629.17829
[8]	validation_0-rmse:860039.20964
[9]	validation_0-rmse:828368.90103
[10]	validation_0-rmse:798476.08492
[11]	validation_0-rmse:770552.77563
[12]	validation_0-rmse:743942.04177
[13]	validation_0-rmse:719332.84291
[14]	validation_0-rmse:696050.29498
[15]	validation_0-rmse:674334.22360
[16]	validation_0-rmse:653999.12453
[17]	validation_0-rmse:634713.55912
[18]	validation_0-rmse:617067.55474
[19]	validation_0-rmse:600456.65184
[20]	validation_0-rmse:585154.95674
[21]	validation_0-rmse:570511.08907
[22]	validation_0-rmse:557257.13236
[23]	validation_0-rmse:544530.72213
[24]	validation_0-rmse:532991.79233
[25]	validation_0-rmse:522224.56455
[26]	validation_0-rmse:512465.95072
[27]	validation_0-rmse:502651.60809
[28]	validation_0-rmse:494266.76465
[29]	validation_0-rmse:485836.45700
[30]	validation_0-rmse:478452.52928
[31]	validation_0-rmse:471711.36327
[32]	validation_0-rmse:465434.72155
[33]	validation_0-rmse:459609.74

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=4, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

# Conclusion

[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a leading software library for working with standard tabular data (the type of data you store in Pandas DataFrames, as opposed to more exotic types of data like images and videos).  With careful parameter tuning, you can train highly accurate models. 

# Your Turn

Train your own model with XGBoost in the **[next exercise](https://www.kaggle.com/kernels/fork/3370271)**!